In [ ]:
%pip install requests-cache
%pip install appdirs
%pip install frozendict
%pip install multitasking

### Import Necessary Packages

In [ ]:
import requests_cache
import yfinance as yf
import pandas as pd
import numpy as np
import tensorflow as tf
import requests
import os
import datetime
import IPython
import IPython.display

from lts import LTSCell
from windowgenerator import WindowGenerator
from baseline import Baseline

### Download Data

In [ ]:
#Get data
data=yf.download("GOOGL", period="1y", interval="1d")
df = pd.DataFrame(data)
df.head()

### Normalize the data

__You can get usable signals by using sine and cosine transforms to clear "Time of year" signals__

In [ ]:
df['Date'] = df.index
x = len(df)
df["num_index"] = range(0, x, 1)
df.set_index(df.pop('num_index'), inplace=True)

date_time = pd.to_datetime(df.pop('Date'), format='%d.%m.%Y %H:%M:%S')

timestamp_s = date_time.map(pd.Timestamp.timestamp)

day = 24*60*60
year = (365.2425)*day

df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [ ]:
#Remove unnecessary columns
df.drop(columns=[
    'Adj Close'],
    inplace=True)

df.head()

In [ ]:
df.info()

### Split the data into training, test, validation datasets.

In [ ]:
column_indices = {name: i for i, name in enumerate(data.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

In [ ]:
timesteps = df.index.to_numpy()
financials = df.to_numpy()

In [ ]:
timesteps, financials

### Normalize the data

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

### Create Window Generator

In [ ]:
single_step_window = WindowGenerator(
    input_width=1, label_width=1, shift=1, train_df=train_df, val_df=val_df, test_df=test_df,
    label_columns=['Close'])
single_step_window

In [ ]:
# Test Baseline

baseline = Baseline(label_index=column_indices['Close'])

baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                 metrics=[tf.keras.metrics.MeanAbsoluteError()])

val_performance = {}
performance = {}
val_performance['Baseline'] = baseline.evaluate(single_step_window.val)
performance['Baseline'] = baseline.evaluate(single_step_window.test, verbose=0)

In [ ]:
# predict 1 step at a time
HORIZON = 1

# use a week worth of timesteps to predict the horizon
WINDOW_SIZE = 7

In [ ]:
w1 = WindowGenerator(
    input_width=WINDOW_SIZE, label_width=WINDOW_SIZE, shift=HORIZON, train_df=train_df, val_df=val_df, test_df=test_df,
    label_columns=['Close'])
w1

In [ ]:
print('Input shape:', w1.example[0].shape)
print('Output shape:', baseline(w1.example[0]).shape)

In [ ]:
w1.plot(baseline)

### Compile and Fit

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(learning_rate = .01),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

### Test with CONV Model

In [ ]:
CONV_WIDTH = 5
conv_window = WindowGenerator(
    input_width=CONV_WIDTH,
    label_width=1,
    shift=1,
    train_df=train_df, 
    val_df=val_df, 
    test_df=test_df,
    label_columns=['Close'])

conv_window

In [ ]:
LABEL_WIDTH = 7
INPUT_WIDTH = LABEL_WIDTH + (CONV_WIDTH - 1)
wide_conv_window = WindowGenerator(
    input_width=INPUT_WIDTH,
    label_width=LABEL_WIDTH,
    shift=1,
    train_df=train_df, 
    val_df=val_df, 
    test_df=test_df,
    label_columns=['Close'])

wide_conv_window

In [ ]:
conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32,
                           kernel_size=(CONV_WIDTH,),
                           activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
])

In [ ]:
print("Conv model on `wide_conv_window`")
print('Input shape:', wide_conv_window.example[0].shape)
print('Output shape:', wide_conv_model(conv_window.example[0]).shape)

In [ ]:
history = compile_and_fit(conv_model, wide_conv_window)

IPython.display.clear_output()
val_performance['Conv'] = conv_model.evaluate(wide_conv_window.val)
performance['Conv'] = conv_model.evaluate(wide_conv_window.test, verbose=0)

In [ ]:
print("Wide window")
print('Input shape:', w1.example[0].shape)
print('Labels shape:', w1.example[1].shape)
print('Output shape:', conv_model(w1.example[0]).shape)

In [ ]:
print("Wide conv window")
print('Input shape:', wide_conv_window.example[0].shape)
print('Labels shape:', wide_conv_window.example[1].shape)
print('Output shape:', conv_model(wide_conv_window.example[0]).shape)

In [ ]:
wide_conv_window.plot(conv_model)

### LSTM Model

In [ ]:
# Model checkpoint with a specific filename
def create_model_checkpoint(model_name, save_path='model_checkpoints', monitor_dataset_loss=False):
    return tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(save_path, model_name),
        verbose=0,
        save_best_only=True,
        #monitor='loss' if monitor_dataset_loss else 'val_loss',
        monitor='loss'
    )

# Create a tensorboard callback
def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + '/' + experiment_name + '/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
    )

    print(f'Saving TensorBoard log files to: {log_dir}')
    return tensorboard_callback

# Create early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=200,
    restore_best_weights=True
)

# create reduce lr on plateau callback
reduce_lr_plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss',
    patience=100,
    verbose=1
)

In [ ]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [ ]:
print('Input shape:', w1.example[0].shape)
print('Output shape:', lstm_model(w1.example[0]).shape)

In [ ]:
history = compile_and_fit(lstm_model, w1)

IPython.display.clear_output()
val_performance['LSTM'] = lstm_model.evaluate(w1.val)
performance['LSTM'] = lstm_model.evaluate(w1.test, verbose=0)

In [ ]:
w1.plot(lstm_model)

In [ ]:
lts_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.RNN(LTSCell(32), time_major=True, return_sequences=True),
    tf.keras.layers.LSTM(32, activation='relu'),
    tf.keras.layers.Dense(HORIZON, activation='linear')
])

In [ ]:
history = compile_and_fit(lts_nodel, w1)

IPython.display.clear_output()
val_performance['LTS'] = lts_model.evaluate(w1.val)
performance['LTS'] = lts_model.evaluate(w1.test, verbose=0)